In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

In [3]:
df = pd.read_csv('heart.csv')
df = df[['age', 'sex', 'resting bp s', 'max heart rate', 'target']]
df.target.value_counts(normalize=True)

target
1    0.528571
0    0.471429
Name: proportion, dtype: float64

In [4]:
df

,age,sex,resting bp s,max heart rate,target
0,40,1,140,172,0
1,49,0,160,156,1
2,37,1,130,98,0
3,48,0,138,108,1
4,54,1,150,122,0
...,...,...,...,...,...
1185,45,1,110,132,1
1186,68,1,144,141,1
1187,57,1,130,115,1
1188,57,0,130,174,1


In [5]:
X = df.drop('target', axis=1).values
y = df['target'].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

In [7]:
model = LogisticRegression().fit(X_train, y_train)

y_pred = model.predict(X_test)

In [8]:
acc = accuracy_score(y_test, y_pred)
print(acc)
prec = precision_score(y_test, y_pred)
print(prec)
rec = recall_score(y_test, y_pred)
print(rec)
rocauc = roc_auc_score(y_test, y_pred)
print(rocauc)

0.7310924369747899
0.75
0.7380952380952381
0.7306547619047619


In [10]:
p = np.array([28, 1, 140, 170]).reshape(1,-1)

In [11]:
model.predict(p.reshape(1,-1))[0]

np.int64(0)

In [12]:
model.predict([[28, 1, 140, 170]])[0]

np.int64(0)

## Save model

In [13]:
import pickle

# save
with open("model_lr.pkl", 'wb') as f:
    pickle.dump(model, f)

# load
with open('model_lr.pkl', 'rb') as f:
    model = pickle.load(f)

# Gradio

In [70]:
# !pip install gradio

In [4]:
import gradio as gr
import pickle

In [9]:
# load
with open('titan_model.pkl', 'rb') as f:
    model = pickle.load(f)

def authenticate(username, password):
    if username == "user" and password == "word":
        return True
    else:
        return False

def greet(Age, Sex, BloodPressure, MaxHeartRate=100, SomeValue=None):
    result = model.predict([[Age, Sex, BloodPressure, MaxHeartRate]])[0]
    if result == 0:
        return 'Вы вне группы сердечного риска'
    else:
        return 'Вы в группе риска, обратитесь к врачу!'

demo = gr.Interface(
    fn=greet,
    inputs=[
        gr.Slider(
            info='Enter your age',
            minimum=1,
            maximum=100,
            value=20,
            step=1
            ),
        gr.Radio(['1', '2'], value=2),
        gr.Slider(
            info='1 - Male, 0 - Female',
            value=120,
            minimum=40,
            maximum=220,
            step=10,
            ),
        gr.Number(
            value=160,
            info='Max heart rate',
        )
        # gr.Checkbox()
    ],
    outputs=[
        gr.Textbox(label='Result', lines=1)
    ],
    additional_inputs=[
        gr.Slider(0,1000)
    ],
    title="Check your heart risk",
    description="Some description",
    # theme=gr.themes.Monochrome()
)

demo.launch(share=False, auth=authenticate)

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
